### Step 1: Import the Packages

In [1]:
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
from google.cloud import bigquery_storage
import warnings
warnings.filterwarnings(action="ignore")

g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


### Step 2.1: Instantiate a BQ client

In [2]:
client = bigquery.Client(project="logistics-data-staging-flat")
bqstorage_client = bigquery_storage.BigQueryReadClient()

### Step 2.2: Import the SQL queries and run them

In [3]:
sql_queries = [i for i in os.listdir(path=os.getcwd() + "/sql_queries") if i.endswith(".sql")]

for i in sql_queries:
    with open(file=os.getcwd() + f"/sql_queries/{i}", mode="r") as sql:
        client.query(query=sql.read()).result()
        sql.close()

### Step 2.3: Pull the order data from the final dataset

In [4]:
df = client\
    .query(query="""SELECT * FROM `dh-logistics-product-ops.pricing.ab_test_individual_orders_cleaned_lb_rollout_tests`""")\
    .result()\
    .to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm")

Downloading: 100%|██████████| 740122/740122 [01:27<00:00, 8421.98rows/s] 


### Step 2.4: Create a dataset of the business KPIs analyzed on the treatment scope level

In [28]:
df.groupby(["region", "entity_id", "test_id", "test_name", "variant"])

,created_date_utc,order_placed_at_utc,order_placed_at_local,dow_local,dps_sessionid_created_at_utc,day_num_in_test,region,entity_id,country_code,city_name,city_id,zone_name,zone_id,zone_shape,customer_location,variant,test_id,test_name,test_start_date,test_end_date,perseus_client_id,ga_session_id,dps_sessionid,dps_customer_tag,customer_total_orders,customer_first_order_date,days_since_first_order,order_id,platform_order_code,scheme_id,vendor_price_scheme_type,vendor_id,target_group,target_group_bi,is_in_treatment,chain_id,chain_name,vertical_type,vendor_vertical_parent,delivery_status,is_own_delivery,exchange_rate,dps_delivery_fee_local,delivery_fee_local,commission_local,joker_vendor_fee_local,service_fee_local,sof_local_cdwh,sof_local,delivery_costs_local,actual_df_paid_by_customer,gfv_local,gmv_local,mean_delay,dps_mean_delay,dps_mean_delay_zone_id,travel_time,dps_travel_time,travel_time_distance_km,delivery_distance_m,to_customer_time,actual_DT,is_delivery_fee_covered_by_discount,is_delivery_fee_covered_by_voucher,parent_vertical_concat,keep_drop_flag,revenue_local,gross_profit_local
0,2022-11-30,2022-11-30 05:30:58+00:00,2022-11-30 13:30:58,Wednesday,2022-11-30 05:30:58.356762+00:00,2,Asia,FP_PH,ph,Cebu,2,Cebu city,2,"POLYGON((123.851623545634 10.2975234328917, 12...",POINT(123.89723 10.34145),Variation3,80.000000000,PH_20221128_LovedBrandsCebu,2022-11-28 09:14:42.417033+00:00,NaT,1601626527165.7770754662.kjbnyvrdgg,None,1669786119583.1936084720.zggamxqafd,None,33.000000000,2020-10-03 05:26:00+00:00,788,337438857,pbvi-npzt,2331.000000000,Experiment,pbvi,TG2,Target Group 2,True,None,None,restaurants,restaurant,completed,True,58.793980000,44.000000000,44.000000000,72.540000000,0E-9,0E-9,0E-9,0E-9,76.610000,39.29,325.000000000,269.000000000,7.949973,7.008485443,2.000000000,11.02,11.019514875,3.002959,4230.0,15.15,29.90,False,False,Restaurant,Keep,111.83,35.220000
1,2022-11-30,2022-11-30 02:59:23+00:00,2022-11-30 10:59:23,Wednesday,2022-11-30 02:59:23.275621+00:00,1,Asia,FP_MY,my,Kedah,213,Jitra,278,"POLYGON((100.38900751964 6.24144439091272, 100...",POINT(100.409449 6.290627),Variation2,37.000000000,MY_20221129_LB_Kedah,2022-11-29 09:43:15.968773+00:00,NaT,1621165455483.579650817133877519.CdkbhvSIpo,None,1669776940387.398608963281075838.SBvvKADQRh,Existing,457.000000000,2020-06-14 05:18:01+00:00,898,406524483,r8lb-xsv4,582.000000000,Automatic scheme,r8lb,Non_TG,None,False,None,None,restaurants,restaurant,completed,True,4.667450000,6.000000000,6.000000000,6.790000000,0E-9,0E-9,0E-9,0E-9,5.321806,6.00,24.000000000,29.000000000,5.461672,3.149432440,278.000000000,9.41,9.409224587,3.600213,4358.0,8.67,25.23,False,False,None,Keep,12.79,7.468194
2,2022-11-29,2022-11-29 04:04:20+00:00,2022-11-29 12:04:20,Tuesday,2022-11-29 04:04:19.964964+00:00,1,Asia,FP_PH,ph,South mm,1,East santa rosa,117,"POLYGON((121.039807385823 14.300725537039, 121...",POINT(121.05714435 14.3139687),Control,83.000000000,PH_20221128_LovedBrandsSouthmm,2022-11-28 09:19:31.478317+00:00,NaT,1611656441624.392614942535477752.SpEpym1DNh,15099642262293479444.1669692738,1669692733335.818101245110688721.r52M6rdUJD,None,None,NaT,<NA>,336988343,fr8l-crwk,2195.000000000,Campaign,fr8l,Non_TG,None,False,cg0ep,Jollibee,restaurants,restaurant,completed,True,58.585390000,33.000000000,33.000000000,0E-9,0E-9,0E-9,0E-9,0E-9,43.770000,33.00,243.000000000,276.000000000,11.023572,10.558606108,117.000000000,2.81,2.806763362,0.764879,1081.0,5.93,21.85,False,False,None,Keep,33.00,-10.770000
3,2022-11-30,2022-11-30 11:23:04+00:00,2022-11-30 19:23:04,Wednesday,2022-11-30 11:23:04.603975+00:00,2,Asia,FP_PH,ph,Pampanga,203,Angeles,18,"POLYGON((120.548601150513 15.1527071576886, 12...",POINT(120.5576539 15.1588806),Variation2,81.000000000,PH_20221128_LovedBrandsPampanga,2022-11-28 09:17:28.691691+00:00,NaT,1640325664276.196950493211978409.Z5IXosQ70i,None,1669807328463.378178017210375066.nMt3ArFT6Q,None,142.000000000,2022-02-13 03:04:09+00:00,290,337614050,wcmy-c407,2342.000000000,Experiment,